In [1]:
import requests
import pandas as pd
import time
import numpy as np
import re
import sys
import random

In [2]:
jeju_tour = pd.read_csv('../data/jeju_tour_places_modified.csv')

In [3]:
jeju_tour.shape

(1103, 25)

In [4]:
jeju_tour.head(2)

,alltag,contentsid,contentscd,title,address,roadaddress,tag,introduction,readcnt,likecnt,...,evelpt,latitude,longitude,phoneno,sbst,PHOTOSHARECOUNT,cate2cd,cate3cd,imgpath,thumbnailpath
0,"휴식/힐링,체험관광,액티비티,사계절",CNTS_200000000012469,관광지,제로스테이션(제로포인트트레일),제주특별자치도 제주시 건입동 1136 2층,제주특별자치도 제주시 산지로 25 2층,"휴식/힐링,체험관광,액티비티,사계절,체험,레저/체험,어트랙션",제로스테이션은 산지천에 위치한 카페이자 길과 여행을 사랑하는 사람들을 위한 공간이다...,530,0,...,0.0,33.514733,126.529280,064-702-1947,"트레킹 프로그램과 관련해 사전 상담과 당일 모니터링, 완주 인증 기록 게시 등 다양...",0.0,레저/체험,그외,https://api.cdn.visitjeju.net/photomng/imgpath...,https://api.cdn.visitjeju.net/photomng/thumbna...
1,"부모,커플,친구,경관/포토,휴식/힐링,테마공원,오름,사계절,청년,중/장년,휴식/치유...",CNTS_200000000009501,관광지,궁대오름(궁대악),제주특별자치도 서귀포시 성산읍 수산리 4711-1,제주특별자치도 서귀포시 성산읍 금백조로 446,"부모,커플,친구,경관/포토,휴식/힐링,테마공원,오름,사계절,청년,중/장년,휴식/치유...","오름의 모양새가 마치 활처럼 생기고, 오름 허리에 활(궁·弓)자 모양의 띠가 둘러져...",1723,0,...,0.0,33.440562,126.822089,064-787-4711,"오름의 모양새가 마치 활처럼 생기고, 오름 허리에 활(궁·弓)자 모양의 띠가 둘러져...",0.0,자연,오름,https://api.cdn.visitjeju.net/photomng/imgpath...,https://api.cdn.visitjeju.net/photomng/thumbna...


In [77]:
# 1차 검색 제주 000 
url = 'https://map.naver.com/v5/api/search'
params = {
    'calller': 'pcweb',
    'query': '',
    'type': 'all',
    'searchCoord': '126.56379330000001;33.24652849999992',
    'page': 1,
    'displayCount': 20,
    'isPlaceRecommendationReplace': True,
    'lang': 'ko'
}
tmp_dict = dict()
headers = {
    'accept': 'application/json, text/plain, */*',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'ko-KR,ko;q=0.8,en-US;q=0.6,en;q=0.4',
    'cache-control': 'no-cache',
    'content-type': 'application/json',
    'cookie': 'NNB=SFVP2LHVPD7WC; NDARK=N; _gcl_au=1.1.876992382.1652928740; _fbp=fb.1.1652928739593.996991756; _ga=GA1.2.1723077753.1655082737; MM_NEW=1; NFS=2; _tt_enable_cookie=1; _ttp=d30a8fad-c361-4bb2-a6d7-884e4f9ca1c6; NV_WETR_LOCATION_RGN_M="MDkxNDAxMDQ="; NV_WETR_LAST_ACCESS_RGN_M="MDkxNDAxMDQ="; nx_ssl=2; csrf_token=b22420dac145f9757a431121f5b4c25948818847853c526728a4d2219155c97e9b6490d088524f5e2d1f9fdd189f23d95bdd86f6e9ca8c8b07bf30c857e040c6; page_uid=d78aa3bb-613a-4eab-8035-b9c4712fa1de',
    'expires': 'Sat, 01 Jan 2000 00:00:00 GMT',
    'pragma': 'no-cache',
    'referer': 'https://map.naver.com/',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'
}

In [6]:
params2 ={
    'lang' :'ko'
}
#n차 검색 제주도 주소 검색 후 해당 위치의 가게 정보를 조회 
url3 = 'https://map.naver.com/v5/api/addresses/place/'
params3= {
    'address':'',
    'lang':'ko',
    'page':1
}
headers2 = {
    'authority': 'map.naver.com',
    'accept': 'application/json, text/plain, */*',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'ko-KR,ko;q=0.8,en-US;q=0.6,en;q=0.4',
    'cache-control': 'no-cache',
    'content-type': 'application/json',
    'cookie': 'NNB=SFVP2LHVPD7WC; NDARK=N; _gcl_au=1.1.876992382.1652928740; _fbp=fb.1.1652928739593.996991756; _ga=GA1.2.1723077753.1655082737; MM_NEW=1; NFS=2; _tt_enable_cookie=1; _ttp=d30a8fad-c361-4bb2-a6d7-884e4f9ca1c6; NV_WETR_LOCATION_RGN_M="MDkxNDAxMDQ="; NV_WETR_LAST_ACCESS_RGN_M="MDkxNDAxMDQ="; nx_ssl=2; page_uid=hXuWqwp0YiRssBM4BUGssssssMo-163853; BMR=s=1658907970093&r=https://m.blog.naver.com/PostView.naver?isHttpsRedirect=true&blogId=heygun&logNo=221520454294&r2=https://www.google.com/; page_uid=3c7d105d-de2b-4c43-8a74-1c2abebf354d',
    'expires': 'Sat, 01 Jan 2000 00:00:00 GMT',
    'pragma ': 'no-cache',
    'referer': 'https://map.naver.com/',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'
}

In [7]:
title = jeju_tour['title']

In [14]:
tour_result = []
none_lst = []
for num,v in enumerate(title):
    searchword = '제주 '+ title[num]
    #정규표현식으로 ( ) 안 텍스트 전부 삭제
    rmve_bracket = "\(.*\)|\s-\s.*"
    searchword = re.sub(rmve_bracket,'',searchword)
    params['query'] = searchword
    time.sleep(2)
    response1 = requests.get(url,params,headers=headers)
    if response1.status_code == 200:
        j_text = response1.json()
    else:
        print(f'{num}번째 실패')
        sys.exit('response1.status_code 조회해보기')
        
    #장소정보 저장    
    tmp_places = j_text['result']['place']    

    #검색결과 없을 때
    if tmp_places is None:
        none_lst.append(title[num])
        print(f'{num} 번째 {title[num]} 조회 결과 존재 X')
        continue
    #검색 결과 존재시    
    else:
        tmp_list=tmp_places['list']
        #검색결과가 하나일 때 바로 조회
        if len(tmp_list) == 1:
            tmp_id = tmp_list[0]['id']
            url2 = 'https://map.naver.com/v5/api/sites/summary/' + tmp_id
            time.sleep(2)
            response2 = requests.get(url2,params2,headers=headers2)
            if response2.status_code == 200:
                tmp_dict = response2.json()
                tour_result.append(tmp_dict)
                print(f'{num}번째 {searchword} 중 {tmp_id}의 여행지 요약본 저장 완료')
            else:
                print(f'{num}번째 {searchword}의 {tmp_id}로 여행지 요약본 가져오기 실패')
                sys.exit('response2.status_code 조회해보기')
            continue
        #검색결과 여러개일 때    
        else:
            for index, lst in enumerate(tmp_list):
                tmp_categories =lst['category']
                for category in tmp_categories:
                    if '여행' in category:
                        tmp_id = lst['id']
                        #주위 가게로 가져온 id 값으로 요약본 출력
                        url2 = 'https://map.naver.com/v5/api/sites/summary/' + tmp_id
                        time.sleep(2)
                        response2 = requests.get(url2,params2,headers=headers2)
                        if response2.status_code == 200:
                            tmp_dict = response2.json()
                            tour_result.append(tmp_dict)
                            print(f'{num}번째 {searchword} 중 {tmp_id}의 여행지 요약본 저장 완료')
                        else:
                            print(f'{num}번째 {searchword}의 {tmp_id}로 여행지 요약본 가져오기 실패')
                            sys.exit('response2.status_code 조회해보기')
                        break
                    elif '기념' in category:
                        tmp_id = lst['id']
                        #주위 가게로 가져온 id 값으로 요약본 출력
                        url2 = 'https://map.naver.com/v5/api/sites/summary/' + tmp_id
                        time.sleep(2)
                        response2 = requests.get(url2,params2,headers=headers2)
                        if response2.status_code == 200:
                            tmp_dict = response2.json()
                            tour_result.append(tmp_dict)
                            print(f'{num}번째 {searchword} 중 {tmp_id}의  기념품 요약본 저장 완료')
                        else:
                            print(f'{num}번째 {searchword}의 {tmp_id}로 기념품 요약본 가져오기 실패')
                            sys.exit('response2.status_code 조회해보기')
                        break
                    elif '지명' in category:
                        tmp_id = lst['id']
                        #주위 가게로 가져온 id 값으로 요약본 출력
                        url2 = 'https://map.naver.com/v5/api/sites/summary/' + tmp_id
                        time.sleep(2)
                        response2 = requests.get(url2,params2,headers=headers2)
                        if response2.status_code == 200:
                            tmp_dict = response2.json()
                            tour_result.append(tmp_dict)
                            print(f'{num}번째 {searchword} 중 {tmp_id}의 지명 요약본 저장 완료')
                        else:
                            print(f'{num}번째 {searchword}의 {tmp_id}로 지명 요약본 가져오기 실패')
                            sys.exit('response2.status_code 조회해보기')
                        break
                    elif '스포츠' in category:
                        tmp_id = lst['id']
                        #주위 가게로 가져온 id 값으로 요약본 출력
                        url2 = 'https://map.naver.com/v5/api/sites/summary/' + tmp_id
                        time.sleep(2)
                        response2 = requests.get(url2,params2,headers=headers2)
                        if response2.status_code == 200:
                            tmp_dict = response2.json()
                            tour_result.append(tmp_dict)
                            print(f'{num}번째 {searchword} 중 {tmp_id}의 스포츠 요약본 저장 완료')
                        else:
                            print(f'{num}번째 {searchword}의 {tmp_id}로 스포츠 요약본 가져오기 실패')
                            sys.exit('response2.status_code 조회해보기')
                        break   

                    elif '공방' in category:
                        tmp_id = lst['id']
                        #주위 가게로 가져온 id 값으로 요약본 출력
                        url2 = 'https://map.naver.com/v5/api/sites/summary/' + tmp_id
                        time.sleep(2)
                        response2 = requests.get(url2,params2,headers=headers2)
                        if response2.status_code == 200:
                            tmp_dict = response2.json()
                            tour_result.append(tmp_dict)
                            print(f'{num}번째 {searchword} 중 {tmp_id}의 공방 요약본 저장 완료')
                        else:
                            print(f'{num}번째 {searchword}의 {tmp_id}로 스포츠 요약본 가져오기 실패')
                            sys.exit('response2.status_code 조회해보기')
                        break
                        
    print(f'{num}번째 {searchword} 종료')

0번째 제주 제로스테이션 중 32152539의 스포츠 요약본 저장 완료
0번째 제주 제로스테이션 중 1744843816의 공방 요약본 저장 완료
0번째 제주 제로스테이션 종료
1번째 제주 궁대오름 중 17078730의 여행지 요약본 저장 완료
2번째 제주 헌마공신김만일기념관 중 1317124374의 여행지 요약본 저장 완료
2번째 제주 헌마공신김만일기념관 종료
3번째 제주 본초족욕 종료
4번째 제주 강병대교회 중 1033492321의 여행지 요약본 저장 완료
5번째 제주 토산봉 중 17089759의 여행지 요약본 저장 완료
6번째 제주 모지오름 중 17088128의 여행지 요약본 저장 완료
6번째 제주 모지오름 종료
7번째 제주 개오름 중 17087735의 여행지 요약본 저장 완료
7번째 제주 개오름 중 16994185의 여행지 요약본 저장 완료
7번째 제주 개오름 중 17084701의 여행지 요약본 저장 완료
7번째 제주 개오름 중 17084537의 지명 요약본 저장 완료
7번째 제주 개오름 종료
8번째 제주 동일리포구 중 17067976의 여행지 요약본 저장 완료
9번째 제주 대정향교 중 12226274의 여행지 요약본 저장 완료
9번째 제주 대정향교 종료
박성내 조회 결과 존재 X


In [17]:
for num in range(10,len(title)):
    searchword = '제주 '+ title[num]
    #정규표현식으로 ( ) 안 텍스트 전부 삭제
    rmve_bracket = "\(.*\)|\s-\s.*"
    searchword = re.sub(rmve_bracket,'',searchword)
    params['query'] = searchword
    time.sleep(2)
    response1 = requests.get(url,params,headers=headers)
    if response1.status_code == 200:
        j_text = response1.json()
    else:
        print(f'{num}번째 실패')
        sys.exit('response1.status_code 조회해보기')
        
    #장소정보 저장    
    tmp_places = j_text['result']['place']    

    #검색결과 없을 때
    if tmp_places is None:
        none_lst.append(title[num])
        print(f'{num} 번째 {title[num]} 조회 결과 존재 X')
        continue
    #검색 결과 존재시    
    else:
        tmp_list=tmp_places['list']
        #검색결과가 하나일 때 바로 조회
        if len(tmp_list) == 1:
            tmp_id = tmp_list[0]['id']
            url2 = 'https://map.naver.com/v5/api/sites/summary/' + tmp_id
            time.sleep(2)
            response2 = requests.get(url2,params2,headers=headers2)
            if response2.status_code == 200:
                tmp_dict = response2.json()
                tour_result.append(tmp_dict)
                print(f'{num}번째 {searchword} 중 {tmp_id}의 여행지 요약본 저장 완료')
            else:
                print(f'{num}번째 {searchword}의 {tmp_id}로 여행지 요약본 가져오기 실패')
                sys.exit('response2.status_code 조회해보기')
            continue
        #검색결과 여러개일 때    
        else:
            for index, lst in enumerate(tmp_list):
                tmp_categories =lst['category']
                for category in tmp_categories:
                    if '여행' in category:
                        tmp_id = lst['id']
                        #주위 가게로 가져온 id 값으로 요약본 출력
                        url2 = 'https://map.naver.com/v5/api/sites/summary/' + tmp_id
                        time.sleep(2)
                        response2 = requests.get(url2,params2,headers=headers2)
                        if response2.status_code == 200:
                            tmp_dict = response2.json()
                            tour_result.append(tmp_dict)
                            print(f'{num}번째 {searchword} 중 {tmp_id}의 여행지 요약본 저장 완료')
                        else:
                            print(f'{num}번째 {searchword}의 {tmp_id}로 여행지 요약본 가져오기 실패')
                            sys.exit('response2.status_code 조회해보기')
                        break
                    elif '기념' in category:
                        tmp_id = lst['id']
                        #주위 가게로 가져온 id 값으로 요약본 출력
                        url2 = 'https://map.naver.com/v5/api/sites/summary/' + tmp_id
                        time.sleep(2)
                        response2 = requests.get(url2,params2,headers=headers2)
                        if response2.status_code == 200:
                            tmp_dict = response2.json()
                            tour_result.append(tmp_dict)
                            print(f'{num}번째 {searchword} 중 {tmp_id}의  기념품 요약본 저장 완료')
                        else:
                            print(f'{num}번째 {searchword}의 {tmp_id}로 기념품 요약본 가져오기 실패')
                            sys.exit('response2.status_code 조회해보기')
                        break
                    elif '지명' in category:
                        tmp_id = lst['id']
                        #주위 가게로 가져온 id 값으로 요약본 출력
                        url2 = 'https://map.naver.com/v5/api/sites/summary/' + tmp_id
                        time.sleep(2)
                        response2 = requests.get(url2,params2,headers=headers2)
                        if response2.status_code == 200:
                            tmp_dict = response2.json()
                            tour_result.append(tmp_dict)
                            print(f'{num}번째 {searchword} 중 {tmp_id}의 지명 요약본 저장 완료')
                        else:
                            print(f'{num}번째 {searchword}의 {tmp_id}로 지명 요약본 가져오기 실패')
                            sys.exit('response2.status_code 조회해보기')
                        break
                    elif '스포츠' in category:
                        tmp_id = lst['id']
                        #주위 가게로 가져온 id 값으로 요약본 출력
                        url2 = 'https://map.naver.com/v5/api/sites/summary/' + tmp_id
                        time.sleep(2)
                        response2 = requests.get(url2,params2,headers=headers2)
                        if response2.status_code == 200:
                            tmp_dict = response2.json()
                            tour_result.append(tmp_dict)
                            print(f'{num}번째 {searchword} 중 {tmp_id}의 스포츠 요약본 저장 완료')
                        else:
                            print(f'{num}번째 {searchword}의 {tmp_id}로 스포츠 요약본 가져오기 실패')
                            sys.exit('response2.status_code 조회해보기')
                        break   

                    elif '공방' in category:
                        tmp_id = lst['id']
                        #주위 가게로 가져온 id 값으로 요약본 출력
                        url2 = 'https://map.naver.com/v5/api/sites/summary/' + tmp_id
                        time.sleep(2)
                        response2 = requests.get(url2,params2,headers=headers2)
                        if response2.status_code == 200:
                            tmp_dict = response2.json()
                            tour_result.append(tmp_dict)
                            print(f'{num}번째 {searchword} 중 {tmp_id}의 공방 요약본 저장 완료')
                        else:
                            print(f'{num}번째 {searchword}의 {tmp_id}로 스포츠 요약본 가져오기 실패')
                            sys.exit('response2.status_code 조회해보기')
                        break
                        
    print(f'{num}번째 {searchword} 종료')

10 번째 박성내 조회 결과 존재 X
11번째 제주 산방산 수국 중 13491591의 여행지 요약본 저장 완료
11번째 제주 산방산 수국 중 13044022의 여행지 요약본 저장 완료
11번째 제주 산방산 수국 중 1539131369의 여행지 요약본 저장 완료
11번째 제주 산방산 수국 종료
12번째 제주 모슬봉  중 17065061의 여행지 요약본 저장 완료
12번째 제주 모슬봉  중 38644345의 여행지 요약본 저장 완료
12번째 제주 모슬봉  중 17064757의 여행지 요약본 저장 완료
12번째 제주 모슬봉  종료
13번째 제주 발자국화석공원 중 1987691673의 여행지 요약본 저장 완료
13번째 제주 발자국화석공원 종료
14번째 제주 마복이오름 중 17085066의 여행지 요약본 저장 완료
15 번째 볼레낭길 조회 결과 존재 X
16번째 제주 명월성지 중 1071488603의 여행지 요약본 저장 완료
17번째 제주 가문이오름 중 17030568의 여행지 요약본 저장 완료
17번째 제주 가문이오름 중 1990070640의 여행지 요약본 저장 완료
17번째 제주 가문이오름 종료
18번째 제주 서귀포층패류화석산지 중 13526015의 여행지 요약본 저장 완료
19번째 제주 서귀포자연휴양림 중 17059754의 여행지 요약본 저장 완료
19번째 제주 서귀포자연휴양림 종료
20번째 제주 제주베스트힐 펜션&글램핑 중 36797503의 여행지 요약본 저장 완료
21번째 제주 요가느림원 중 36771178의 여행지 요약본 저장 완료
22 번째 프랑스샤르망 조회 결과 존재 X
23번째 제주 빠우사 중 1103132083의 여행지 요약본 저장 완료
24 번째 바늘오름 조회 결과 존재 X
25번째 제주 제주항일기념관 중 13293501의 여행지 요약본 저장 완료
25번째 제주 제주항일기념관 중 1494975287의 여행지 요약본 저장 완료
25번째 제주 제주항일기념관 종료
26번째 제주 김택화미술관 중 1802071455의 여행지 요약본 저장 완료
27번째 제주 

130번째 제주 카카오본사 중 31234634의 스포츠 요약본 저장 완료
130번째 제주 카카오본사 종료
131번째 제주 우도잠수함 중 17076195의 여행지 요약본 저장 완료
131번째 제주 우도잠수함 종료
132번째 제주 소인국테마파크 중 11555793의 여행지 요약본 저장 완료
132번째 제주 소인국테마파크 중 17084064의 여행지 요약본 저장 완료
132번째 제주 소인국테마파크 종료
133번째 제주 저지 예술인 마을 중 17040509의 여행지 요약본 저장 완료
133번째 제주 저지 예술인 마을 중 1336160226의 스포츠 요약본 저장 완료
133번째 제주 저지 예술인 마을 종료
134번째 제주 문섬 중 12226229의 여행지 요약본 저장 완료
134번째 제주 문섬 중 17044454의  기념품 요약본 저장 완료
134번째 제주 문섬 중 1309077951의 스포츠 요약본 저장 완료
134번째 제주 문섬 중 1931001790의 스포츠 요약본 저장 완료
134번째 제주 문섬 종료
135번째 제주 큰노꼬메오름 중 17030080의 여행지 요약본 저장 완료
136번째 제주 북촌포구 중 1213904047의 여행지 요약본 저장 완료
136번째 제주 북촌포구 종료
137번째 제주 제주바다체험장 중 12995388의 여행지 요약본 저장 완료
138번째 제주 서귀다원 종료
139번째 제주 질그랭이센터 중 1241932478의 여행지 요약본 저장 완료
139번째 제주 질그랭이센터 종료
140번째 제주 삼양해안도로 중 13481077의 여행지 요약본 저장 완료
141번째 제주 물영아리오름 중 17071747의 여행지 요약본 저장 완료
141번째 제주 물영아리오름 중 17089016의 여행지 요약본 저장 완료
141번째 제주 물영아리오름 종료
142번째 제주 신흥2리 동백마을 중 442102892의 여행지 요약본 저장 완료
142번째 제주 신흥2리 동백마을 중 1440012602의 여행지 요약본 저장 완료
142번째 제주 신흥2리 동백마을 종료
143번째 

227번째 제주 샤이니숲길 중 1690179903의 여행지 요약본 저장 완료
228번째 제주 동백동산 중 13481079의 여행지 요약본 저장 완료
228번째 제주 동백동산 중 1820620418의 여행지 요약본 저장 완료
228번째 제주 동백동산 중 12047800의 여행지 요약본 저장 완료
228번째 제주 동백동산 중 11491279의 여행지 요약본 저장 완료
228번째 제주 동백동산 종료
229번째 제주 머체왓숲길 중 38704284의 여행지 요약본 저장 완료
229번째 제주 머체왓숲길 종료
230번째 제주 제주아트서커스 종료
231번째 제주 신화테마파크 중 1870600980의 여행지 요약본 저장 완료
231번째 제주 신화테마파크 중 20947594의 여행지 요약본 저장 완료
231번째 제주 신화테마파크 종료
232번째 제주 용머리해안 중 13491455의 여행지 요약본 저장 완료
232번째 제주 용머리해안 중 1395039246의 여행지 요약본 저장 완료
232번째 제주 용머리해안 중 21251675의 여행지 요약본 저장 완료
232번째 제주 용머리해안 종료
233번째 제주 테디베어뮤지엄제주 중 11607804의 여행지 요약본 저장 완료
233번째 제주 테디베어뮤지엄제주 종료
234번째 제주 빌레못동굴 중 1245460598의 여행지 요약본 저장 완료
234번째 제주 빌레못동굴 중 13542768의 여행지 요약본 저장 완료
234번째 제주 빌레못동굴 종료
235번째 제주 제주빅볼랜드 중 36960447의 여행지 요약본 저장 완료
235번째 제주 제주빅볼랜드 종료
236번째 제주 일출랜드 중 11491279의 여행지 요약본 저장 완료
236번째 제주 일출랜드 중 30820682의 여행지 요약본 저장 완료
236번째 제주 일출랜드 종료
237번째 제주 캔디원 중 1241487322의 여행지 요약본 저장 완료
238번째 제주 서프라이즈테마파크 중 1193663990의 여행지 요약본 저장 완료
238번째 제주 서프라이즈테마파크 종료
239번째 제주 쇠소깍 중

346번째 제주 제주글램핑  종료
347 번째 관음사 군 주둔지 옛터 조회 결과 존재 X
348 번째 9연대 본부 옛터 조회 결과 존재 X
349번째 제주 남문지못 중 1052294895의 여행지 요약본 저장 완료
350번째 제주 수월이못 중 1909427659의 여행지 요약본 저장 완료
351 번째 한남의숙터 조회 결과 존재 X
352 번째 동계 정온 유허비 조회 결과 존재 X
353 번째 천주교길 정난주길(고통의길) 조회 결과 존재 X
354번째 제주 제주아트센터 중 31093439의 여행지 요약본 저장 완료
354번째 제주 제주아트센터 중 79969166의 여행지 요약본 저장 완료
354번째 제주 제주아트센터 중 402964481의 여행지 요약본 저장 완료
354번째 제주 제주아트센터 종료
355번째 제주 장생의 숲길 중 13481075의 여행지 요약본 저장 완료
356번째 제주 캐릭파크 종료
357 번째 문화예술마을 저지리 농어촌체험마을 조회 결과 존재 X
358 번째 우도봉 풍경 조회 결과 존재 X
359번째 제주 팜핑제주 종료
360번째 제주 가세오름 중 17089659의 여행지 요약본 저장 완료
360번째 제주 가세오름 중 16998802의 여행지 요약본 저장 완료
360번째 제주 가세오름 종료
361 번째 동산농원 딸기체험장 조회 결과 존재 X
362번째 제주 우도올레보트 중 37868338의 스포츠 요약본 저장 완료
362번째 제주 우도올레보트 중 1775300701의 여행지 요약본 저장 완료
362번째 제주 우도올레보트 종료
363번째 제주 민오름 중 16998275의 여행지 요약본 저장 완료
363번째 제주 민오름 중 33861025의 여행지 요약본 저장 완료
363번째 제주 민오름 중 17030561의 여행지 요약본 저장 완료
363번째 제주 민오름 중 17071622의 여행지 요약본 저장 완료
363번째 제주 민오름 중 17037335의 여행지 요약본 저장 완료
363번째 제주 민오름 종료
364번째 제주 추사 유배길 종료
365번째 제

483번째 제주 문서프 중 37746343의 여행지 요약본 저장 완료
484 번째 정의현 일관헌 조회 결과 존재 X
485번째 제주 이중섭거주지 중 13476035의 여행지 요약본 저장 완료
486번째 제주 조천비석거리 중 17034450의 여행지 요약본 저장 완료
486번째 제주 조천비석거리 종료
487 번째 WE호텔 웰니스센터 조회 결과 존재 X
488 번째 감귤박물관 월라봉 산책로 - 생이소리길 조회 결과 존재 X
489번째 제주 옹포천 중 1933360519의 스포츠 요약본 저장 완료
489번째 제주 옹포천 중 19651861의 지명 요약본 저장 완료
489번째 제주 옹포천 종료
490번째 제주 예절이오름 중 17072833의 여행지 요약본 저장 완료
491번째 제주 무릉도원마을 중 13109923의 여행지 요약본 저장 완료
491번째 제주 무릉도원마을 중 17068663의 지명 요약본 저장 완료
491번째 제주 무릉도원마을 중 17068694의 여행지 요약본 저장 완료
491번째 제주 무릉도원마을 종료
492번째 제주 플레이박스VR 중 1158542754의 여행지 요약본 저장 완료
493번째 제주 노루생태관찰원 중 13072135의 여행지 요약본 저장 완료
493번째 제주 노루생태관찰원 중 543017588의 여행지 요약본 저장 완료
493번째 제주 노루생태관찰원 종료
494 번째 제주샘주 영농협동조합 조회 결과 존재 X
495번째 제주 자리왓 중 16805113의 여행지 요약본 저장 완료
496 번째 한국건강관리협회제주특별자치도부 조회 결과 존재 X
497번째 제주 돔박이오름  중 1459311247의 여행지 요약본 저장 완료
498번째 제주 평대해변 중 1009102498의 여행지 요약본 저장 완료
498번째 제주 평대해변 종료
499번째 제주 비양도의 비양나무자생지 중 1732640252의 여행지 요약본 저장 완료
500 번째 다끄네포구(용담포구) 조회 결과 존재 X
501 번째 제주기독교순례길 조회 결과 존재 X
502번째 제주 추억애거리 중 

592번째 제주 제주올레 14코스 중 1016124443의 여행지 요약본 저장 완료
592번째 제주 제주올레 14코스 중 17065785의 여행지 요약본 저장 완료
592번째 제주 제주올레 14코스 종료
593번째 제주 법화사지 중 17062402의 여행지 요약본 저장 완료
593번째 제주 법화사지 종료
594번째 제주 초콜릿박물관 중 32165845의 여행지 요약본 저장 완료
594번째 제주 초콜릿박물관 중 16933121의 여행지 요약본 저장 완료
594번째 제주 초콜릿박물관 중 17067640의 여행지 요약본 저장 완료
594번째 제주 초콜릿박물관 종료
595번째 제주 별도연대 중 1316755536의 여행지 요약본 저장 완료
596번째 제주 한화리조트 테라피센터 중 520322049의 여행지 요약본 저장 완료
597번째 제주 북촌국민학교 중 12176998의 여행지 요약본 저장 완료
598번째 제주 북촌환해장성 중 17036262의 여행지 요약본 저장 완료
599 번째 불카분낭 (불타버린 나무) 조회 결과 존재 X
600번째 제주 레더한 중 35489449의 여행지 요약본 저장 완료
601번째 제주 청수곶자왈 중 17038145의 지명 요약본 저장 완료
601번째 제주 청수곶자왈 중 724220777의 여행지 요약본 저장 완료
601번째 제주 청수곶자왈 종료
602번째 제주 곶자왈 중 37232778의 여행지 요약본 저장 완료
602번째 제주 곶자왈 중 13481079의 여행지 요약본 저장 완료
602번째 제주 곶자왈 중 36992417의 여행지 요약본 저장 완료
602번째 제주 곶자왈 중 1385305363의 여행지 요약본 저장 완료
602번째 제주 곶자왈 중 1518633268의 여행지 요약본 저장 완료
602번째 제주 곶자왈 중 19654321의 여행지 요약본 저장 완료
602번째 제주 곶자왈 중 1994658841의 여행지 요약본 저장 완료
602번째 제주 곶자왈 중 1880206452의 여행지 요약본 저장 완료
602번째 제주 곶자왈 중 373

705 번째 제주불교성지 순례길 절로 가는길 - 지계의 길 조회 결과 존재 X
706 번째 표선세화해안도로 조회 결과 존재 X
707번째 제주 이니스프리제주하우스 중 31887096의 여행지 요약본 저장 완료
708번째 제주 거믄여해안경승지 중 17057620의 여행지 요약본 저장 완료
709번째 제주 월라사 중 16922145의 여행지 요약본 저장 완료
710 번째 제주올레 15-A코스 조회 결과 존재 X
711번째 제주 취다선리조트 종료
712 번째 제주불교성지 순례길 절로가는길 - 선정의 길 조회 결과 존재 X
713번째 제주 해품은체험농장 중 35652653의 여행지 요약본 저장 완료
714번째 제주 푸른콩 방주영농조합법인 종료
715번째 제주 제주올레 8코스 중 1623557199의 여행지 요약본 저장 완료
715번째 제주 제주올레 8코스 종료
716 번째 포도갤러리 in 포도호텔 조회 결과 존재 X
717번째 제주 제주올레 7코스 중 1861658580의 여행지 요약본 저장 완료
717번째 제주 제주올레 7코스 중 1421949544의 여행지 요약본 저장 완료
717번째 제주 제주올레 7코스 종료
718번째 제주 제주올레 9코스 중 1421949544의 여행지 요약본 저장 완료
718번째 제주 제주올레 9코스 종료
719번째 제주 하모씨워킹 중 36665687의 여행지 요약본 저장 완료
719번째 제주 하모씨워킹 중 44960766의 여행지 요약본 저장 완료
719번째 제주 하모씨워킹 중 87085257의 여행지 요약본 저장 완료
719번째 제주 하모씨워킹 종료
720번째 제주 송악카트체험장 중 13521131의 여행지 요약본 저장 완료
721번째 제주 귀덕연대 중 1972933478의 여행지 요약본 저장 완료
722번째 제주 제주다이브 체험다이빙 중 1670651626의 스포츠 요약본 저장 완료
722번째 제주 제주다이브 체험다이빙 중 11549238의 스포츠 요약본 저장 완료
722번째 제주 제주다이브 체험다이빙 중 36663137의 스포츠 요약

816번째 제주 제주올레 1-1코스 중 1673944669의 여행지 요약본 저장 완료
816번째 제주 제주올레 1-1코스 중 1612872622의 여행지 요약본 저장 완료
816번째 제주 제주올레 1-1코스 중 1788220830의 여행지 요약본 저장 완료
816번째 제주 제주올레 1-1코스 중 1682001005의 여행지 요약본 저장 완료
816번째 제주 제주올레 1-1코스 중 1147356919의 여행지 요약본 저장 완료
816번째 제주 제주올레 1-1코스 중 1064566529의 여행지 요약본 저장 완료
816번째 제주 제주올레 1-1코스 종료
817번째 제주 제주올레 19코스 중 1788220830의 여행지 요약본 저장 완료
817번째 제주 제주올레 19코스 중 1161824335의 여행지 요약본 저장 완료
817번째 제주 제주올레 19코스 종료
818번째 제주 제주 더팔팔농장 중 1215209534의 여행지 요약본 저장 완료
819 번째 임문숙 가족 헛묘 조회 결과 존재 X
820번째 제주 마오름 중 17040370의 여행지 요약본 저장 완료
821 번째 아이리스촬영지(하얀등대) 조회 결과 존재 X
822 번째 남내소 조회 결과 존재 X
823 번째 조천중학원 옛터 조회 결과 존재 X
824 번째 현의합장묘역 조회 결과 존재 X
825번째 제주 에버리스 골프리조트 종료
826번째 제주 스쿠버스쿨 중 11733509의 스포츠 요약본 저장 완료
826번째 제주 스쿠버스쿨 중 1965568775의 스포츠 요약본 저장 완료
826번째 제주 스쿠버스쿨 중 1658196391의 스포츠 요약본 저장 완료
826번째 제주 스쿠버스쿨 중 1624796932의 스포츠 요약본 저장 완료
826번째 제주 스쿠버스쿨 중 1952827498의 스포츠 요약본 저장 완료
826번째 제주 스쿠버스쿨 종료
827번째 제주 토종닭유통특구교래삼다수마을 중 1914950962의 여행지 요약본 저장 완료
828번째 제주 제주대학교병원 종료
829번째 제주 그제바람 윈드서핑패들보드 중 144

938번째 제주 섶섬 중 1408115948의 스포츠 요약본 저장 완료
938번째 제주 섶섬 종료
939번째 제주 개오리오름 중 17009101의 여행지 요약본 저장 완료
940번째 제주 1100고지습지 중 17063361의 여행지 요약본 저장 완료
941번째 제주 금악오름 중 17020866의 여행지 요약본 저장 완료
941번째 제주 금악오름 중 1417948391의 여행지 요약본 저장 완료
941번째 제주 금악오름 중 17020576의 여행지 요약본 저장 완료
941번째 제주 금악오름 중 1408145013의 여행지 요약본 저장 완료
941번째 제주 금악오름 중 17020893의 여행지 요약본 저장 완료
941번째 제주 금악오름 중 17020659의 여행지 요약본 저장 완료
941번째 제주 금악오름 중 17020731의 여행지 요약본 저장 완료
941번째 제주 금악오름 중 17084770의 여행지 요약본 저장 완료
941번째 제주 금악오름 중 17021000의 여행지 요약본 저장 완료
941번째 제주 금악오름 중 1837219835의 여행지 요약본 저장 완료
941번째 제주 금악오름 중 1741824687의 스포츠 요약본 저장 완료
941번째 제주 금악오름 종료
942 번째 덕천리 팔자 좋아 길 조회 결과 존재 X
943번째 제주 해피맘하우스 종료
944 번째 기독교순례길 1코스(순종의길) 조회 결과 존재 X
945번째 제주 가마오름 진지동굴 중 1580095947의 여행지 요약본 저장 완료
945번째 제주 가마오름 진지동굴 중 17037828의 여행지 요약본 저장 완료
945번째 제주 가마오름 진지동굴 종료
946번째 제주 상추자항 중 1967030336의 여행지 요약본 저장 완료
946번째 제주 상추자항 중 1881359025의 여행지 요약본 저장 완료
946번째 제주 상추자항 종료
947번째 제주 대왕산  중 17079116의 여행지 요약본 저장 완료
947번째 제주 대왕산  중 36143165의 지명 요약본 저장 완료
947번째 제주 대왕산  종료


1043번째 제주 목장카페 드르쿰다 중 19646201의 스포츠 요약본 저장 완료
1043번째 제주 목장카페 드르쿰다 종료
1044번째 제주 안덕계곡 중 11491209의 여행지 요약본 저장 완료
1044번째 제주 안덕계곡 중 17082520의 여행지 요약본 저장 완료
1044번째 제주 안덕계곡 중 17082239의 지명 요약본 저장 완료
1044번째 제주 안덕계곡 중 13332224의 여행지 요약본 저장 완료
1044번째 제주 안덕계곡 중 37784004의 지명 요약본 저장 완료
1044번째 제주 안덕계곡 중 37784000의 지명 요약본 저장 완료
1044번째 제주 안덕계곡 종료
1045번째 제주 옥돔마을 중 1359549644의 여행지 요약본 저장 완료
1046번째 제주 산지등대 종료
1047번째 제주 협재해수욕장 중 11491807의 여행지 요약본 저장 완료
1047번째 제주 협재해수욕장 종료
1048번째 제주 본태박물관 중 31346905의 여행지 요약본 저장 완료
1048번째 제주 본태박물관 종료
1049번째 제주 문도지오름  중 17020576의 여행지 요약본 저장 완료
1049번째 제주 문도지오름  중 1837219835의 여행지 요약본 저장 완료
1049번째 제주 문도지오름  종료
1050번째 제주 조은승마장 중 32420538의 여행지 요약본 저장 완료
1051번째 제주 웃밤 중 17037302의 여행지 요약본 저장 완료
1052번째 제주 광치기해변 중 38466341의 여행지 요약본 저장 완료
1052번째 제주 광치기해변 중 1106408464의 여행지 요약본 저장 완료
1052번째 제주 광치기해변 종료
1053번째 제주 낙천아홉굿마을 의자공원 중 12453098의 여행지 요약본 저장 완료
1054번째 제주 아이스뮤지엄 중 30811752의 여행지 요약본 저장 완료
1054번째 제주 아이스뮤지엄 종료
1055번째 제주 송악산 중 11491447의 여행지 요약본 저장 완료
1055번째 제주 송악산 중 17065134의 여행지 요약본 저장 완료

In [18]:
test1 = pd.DataFrame(tour_result)

In [36]:
none_df =jeju_tour[jeju_tour['title'].isin(none_lst)]

In [37]:
none_df['roadaddress'].isnull().sum()

6

In [43]:
none_df[none_df['roadaddress'].isnull()]['address']

288    제주특별자치도 제주시 추자면 묵리 산 131-1
290                           NaN
297       제주특별자치도 서귀포시 남원읍 수망리 59
426      제주특별자치도 제주시 조천읍 남조로 1660
579                           NaN
775                           NaN
Name: address, dtype: object

In [45]:
none_df.loc[none_df[none_df['roadaddress'].isnull()]['address'].index,'roadaddress'] = none_df[none_df['roadaddress'].isnull()]['address']

In [50]:
none_df.dropna(subset='roadaddress',inplace=True)

C:\Users\TJ\AppData\Local\Temp\ipykernel_15844\4262572759.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  none_df.dropna(subset='roadaddress',inplace=True)


In [59]:
for v in none_df['roadaddress']:
    print(v)

제주특별자치도 제주시 기자길 5
제주특별자치도 서귀포시 안덕면 감산리 1049
제주시 한림읍 명상로 37
제주특별자치도 제주시 조천읍 교래리 산 108
제주특별자치도 서귀포시 표선면 표선당포로
제주특별자치도 서귀포시 성산읍 난산리 2302
제주특별자치도 서귀포시 표선면 번영로 2650
제주특별자치도 서귀포시 대정읍 가파리 642
제주 제주시 한림읍 귀덕11길 19
제주특별자치도 제주시 애월읍 산록서로 15
제주특별자치도 서귀포시 정방동
제주특별자치도서귀포시 서귀동 961-33
제주특별자치도 제주시 아라동 산 24-2
제주특별자치도 서귀포시 성산읍 신천리
제주특별자치도 제주시 애월읍 납읍리 14 57
제주특별자치도 제주시 조천읍 선교로 266
제주특별자치도 제주시 1100로 2894-72
제주특별자치도 제주시 애월읍 광령평화2길 1
제주특별자치도 제주시 한경면 용수1길 108
제주특별자치도 서귀포시 성산읍 서성일로1168번길 89-17
제주특별자치도 제주시 조천읍 남조로 1904
제주특별자치도 서귀포시 안덕면 화순리 776-8
제주특별자치도 서귀포시 강정통물로 (강정동)
제주특별자치도 제주시 애월읍 수산곰솔길 56
제주특별자치도 서귀포시 남원읍 중산간동로 7413
제주특별자치도 제주시 한림읍 상한로
제주특별자치도 제주시 추자면 묵리 산 131-1
제주특별자치도 제주시 한림읍 갯거리오름길 114
제주특별자치도 제주시 추자면 영흥리
제주특별자치도 서귀포시 성산읍 섭지코지로 107 휘닉스제주아일랜드리조트 블루동 1층
제주특별자치도 서귀포시 남원읍 수망리 59
제주특별자치도
제주특별자치도 서귀포시 천제연로 149
제주특별자치도 서귀포시 이어도로 685-1
제주특별자치도 서귀포시 남원읍 서성로 755
제주시 애월읍 광령남6길 54 백제사
제주특별자치도 제주시 산록북로 660
제주특별자치도 서귀포시 대정읍 일주서로 2497
제주특별자치도 서귀포시 대정읍 추사로
제주특별자치도 서귀포시 대정읍 안성리
제주특별자치도 서귀포시 대정읍 추사로247번길 102
제주시 한경면 중산간서

### 이름으로 조회실패한 정보들 주소로 정보 조회

In [65]:
for v in none_df['roadaddress']:
    searchword = v
    #정규표현식으로 0000구 000로 (00동)형식에서 (00) 삭제
    rmve_bracket = "\(.*\)|\s-\s.*"
    searchword = re.sub(rmve_bracket,'',searchword)
    params['query'] = searchword
    time.sleep(2)
    response1 = requests.get(url,params,headers=headers2)
    if response1.status_code == 200:
        j_text = response1.json()
    else: 
        print(f'{num}번째 실패')
        sys.exit('response1.status_code 조회해보기')
    address = j_text['result']['address']
    if address is None:
        none_lst.append(searchword)
        print(f'{v} 검색결과는 존재하지않음')
    else:
        roadAddress = address['roadAddress']
        if roadAddress is None:
            search_address = address['jibunsAddress']['list'][0]['fullAddress']
        else:
            search_address= address['roadAddress']['list'][0]['fullAddress']
            
        params3['address'] = search_address
        page=1
        params3['page'] = page
        time.sleep(2)
        #주위 장소 리스트 호출
        response2 = requests.get(url3,params3,headers=headers)
        if response2.status_code == 200:
            j_text2 = response2.json()
        elif response2.status_code == 404:
            print(f'{v} 주위장소 리스트 존재 X')
            continue
        else:
            print(f'{v} 주위장소 리스트 호출 실패')
            sys.exit('response2.status_code 조회해보기')
        max_page = 20
        total_pages = j_text2['place']['totalCount']
        while max_page * page < total_pages:
            page += 1
            params3['page'] = page
            lsts=j_text2['place']['list']
            for lst in lsts:
                cats = lst['category']
                for cat in cats:
                    #관광지 있는지 조회    
                    if '여행' in cat:
                        tmp_id = lst['id']
                        #주위 가게로 가져온 id 값으로 요약본 출력
                        url2 = 'https://map.naver.com/v5/api/sites/summary/' + tmp_id
                        time.sleep(2)
                        response3 = requests.get(url2,params2,headers=headers2)
                        if response3.status_code == 200:
                            tmp_dict = response3.json()
                            tour_result.append(tmp_dict)
                            print(f'{v}의 여행지 요약본 저장 완료')
                        else:
                            print(f'{v}로 여행지 요약본 가져오기 실패')
                            sys.exit('response3.status_code 조회해보기')
                        break
                    elif '기념' in cat:
                        tmp_id = lst['id']
                        #주위 가게로 가져온 id 값으로 요약본 출력
                        url2 = 'https://map.naver.com/v5/api/sites/summary/' + tmp_id
                        time.sleep(2)
                        response3 = requests.get(url2,params2,headers=headers2)
                        if response3.status_code == 200:
                            tmp_dict = response3.json()
                            tour_result.append(tmp_dict)
                            print(f'{v} 중 {tmp_id}의  기념품 요약본 저장 완료')
                        else:
                            print(f'{v}로 기념품 요약본 가져오기 실패')
                            sys.exit('response3.status_code 조회해보기')
                        break
                    elif '지명' in cat:
                        tmp_id = lst['id']
                        #주위 가게로 가져온 id 값으로 요약본 출력
                        url2 = 'https://map.naver.com/v5/api/sites/summary/' + tmp_id
                        time.sleep(2)
                        response3 = requests.get(url2,params2,headers=headers2)
                        if response3.status_code == 200:
                            tmp_dict = response3.json()
                            tour_result.append(tmp_dict)
                            print(f'{v} 중 {tmp_id}의 지명 요약본 저장 완료')
                        else:
                            print(f'{v}로 지명 요약본 가져오기 실패')
                            sys.exit('response3.status_code 조회해보기')
                        break
                    elif '스포츠' in cat:
                        tmp_id = lst['id']
                        #주위 가게로 가져온 id 값으로 요약본 출력
                        url2 = 'https://map.naver.com/v5/api/sites/summary/' + tmp_id
                        time.sleep(2)
                        response3 = requests.get(url2,params2,headers=headers2)
                        if response3.status_code == 200:
                            tmp_dict = response3.json()
                            tour_result.append(tmp_dict)
                            print(f'{v} 중 {tmp_id}의 스포츠 요약본 저장 완료')
                        else:
                            print(f'{v}로 스포츠 요약본 가져오기 실패')
                            sys.exit('response3.status_code 조회해보기')
                        break
                        
                    elif '공방' in cat:
                        tmp_id = lst['id']
                        #주위 가게로 가져온 id 값으로 요약본 출력
                        url2 = 'https://map.naver.com/v5/api/sites/summary/' + tmp_id
                        time.sleep(2)
                        response3 = requests.get(url2,params2,headers=headers2)
                        if response3.status_code == 200:
                            tmp_dict = response3.json()
                            tour_result.append(tmp_dict)
                            print(f'{v} 중 {tmp_id}의 스포츠 요약본 저장 완료')
                        else:
                            print(f'{v}로 스포츠 요약본 가져오기 실패')
                            sys.exit('response3.status_code 조회해보기')
                        break                        
                                       
            #새로운페이지 불러옴
            time.sleep(2)
            response2 = requests.get(url3,params3,headers=headers)
            if response2.status_code == 200:
                j_text2 = response2.json()
                current_page = j_text2['place']['count']
            else:
                print(f'{v}의 장소 {page} 페이지 출력실패')
                sys.exit('response2.status_code 조회해보기')
        #while문 나온뒤 마지막 페이지 또는 최초 실행 구문
        lsts=j_text2['place']['list']
        for lst in lsts:
            cats = lst['category']
            for cat in cats:
                #관광지 있는지 조회    
                if '여행' in cat:
                    tmp_id = lst['id']
                    #주위 가게로 가져온 id 값으로 요약본 출력
                    url2 = 'https://map.naver.com/v5/api/sites/summary/' + tmp_id
                    time.sleep(2)
                    response3 = requests.get(url2,params2,headers=headers2)
                    if response3.status_code == 200:
                        tmp_dict = response3.json()
                        tour_result.append(tmp_dict)
                        print(f'{v} 중 {tmp_id}의 여행지 요약본 저장 완료')
                    else:
                        print(f'{v}로 여행지 요약본 가져오기 실패')
                        sys.exit('response3.status_code 조회해보기')
                elif '기념' in cat:
                    tmp_id = lst['id']
                    #주위 가게로 가져온 id 값으로 요약본 출력
                    url2 = 'https://map.naver.com/v5/api/sites/summary/' + tmp_id
                    time.sleep(2)
                    response3 = requests.get(url2,params2,headers=headers2)
                    if response3.status_code == 200:
                        tmp_dict = response3.json()
                        tour_result.append(tmp_dict)
                        print(f'{v} 중 {tmp_id}의 기념품 요약본 저장 완료')
                    else:
                        print(f'{v}로 기념품 요약본 가져오기 실패')
                        sys.exit('response3.status_code 조회해보기')
                elif '지명' in cat:
                    tmp_id = lst['id']
                    #주위 가게로 가져온 id 값으로 요약본 출력
                    url2 = 'https://map.naver.com/v5/api/sites/summary/' + tmp_id
                    time.sleep(2)
                    response3 = requests.get(url2,params2,headers=headers2)
                    if response3.status_code == 200:
                        tmp_dict = response3.json()
                        tour_result.append(tmp_dict)
                        print(f'{v} 중 {tmp_id}의 지명 요약본 저장 완료')
                    else:
                        print(f'{v}로 지명 요약본 가져오기 실패')
                        sys.exit('response3.status_code 조회해보기')
                elif '스포츠' in cat:
                    tmp_id = lst['id']
                    #주위 가게로 가져온 id 값으로 요약본 출력
                    url2 = 'https://map.naver.com/v5/api/sites/summary/' + tmp_id
                    time.sleep(2)
                    response3 = requests.get(url2,params2,headers=headers2)
                    if response3.status_code == 200:
                        tmp_dict = response3.json()
                        tour_result.append(tmp_dict)
                        print(f'{v} 의 스포츠 요약본 저장 완료')
                    else:
                        print(f'{v}로 스포츠 요약본 가져오기 실패')
                        sys.exit('response3.status_code 조회해보기')
                    break
                elif '공방' in cat:
                    tmp_id = lst['id']
                    #주위 가게로 가져온 id 값으로 요약본 출력
                    url2 = 'https://map.naver.com/v5/api/sites/summary/' + tmp_id
                    time.sleep(2)
                    response3 = requests.get(url2,params2,headers=headers2)
                    if response3.status_code == 200:
                        tmp_dict = response3.json()
                        tour_result.append(tmp_dict)
                        print(f'{v} 의 공방 요약본 저장 완료')
                    else:
                        print(f'{v}로 공방 요약본 가져오기 실패' )
                        sys.exit('response3.status_code 조회해보기')
                    break                                   
        print(f'{v}완료')

제주특별자치도 제주시 기자길 5 주위장소 리스트 존재 X
제주특별자치도 서귀포시 안덕면 감산리 1049 주위장소 리스트 존재 X
제주시 한림읍 명상로 37 주위장소 리스트 존재 X
제주특별자치도 제주시 조천읍 교래리 산 108 중 17037218의 여행지 요약본 저장 완료
제주특별자치도 제주시 조천읍 교래리 산 108완료
제주특별자치도 서귀포시 표선면 표선당포로 검색결과는 존재하지않음
제주특별자치도 서귀포시 성산읍 난산리 2302 중 17081004의 여행지 요약본 저장 완료
제주특별자치도 서귀포시 성산읍 난산리 2302완료
제주특별자치도 서귀포시 표선면 번영로 2650 중 13040524의 여행지 요약본 저장 완료
제주특별자치도 서귀포시 표선면 번영로 2650 의 스포츠 요약본 저장 완료
제주특별자치도 서귀포시 표선면 번영로 2650 의 스포츠 요약본 저장 완료
제주특별자치도 서귀포시 표선면 번영로 2650완료
제주특별자치도 서귀포시 대정읍 가파리 642 주위장소 리스트 존재 X
제주 제주시 한림읍 귀덕11길 19 주위장소 리스트 존재 X
제주특별자치도 제주시 애월읍 산록서로 15 중 36960447의 여행지 요약본 저장 완료
제주특별자치도 제주시 애월읍 산록서로 15 의 스포츠 요약본 저장 완료
제주특별자치도 제주시 애월읍 산록서로 15 의 스포츠 요약본 저장 완료
제주특별자치도 제주시 애월읍 산록서로 15완료
제주특별자치도 서귀포시 정방동완료
제주특별자치도서귀포시 서귀동 961-33 주위장소 리스트 존재 X
제주특별자치도 제주시 아라동 산 24-2 주위장소 리스트 존재 X
제주특별자치도 서귀포시 성산읍 신천리 중 38162822의 지명 요약본 저장 완료
제주특별자치도 서귀포시 성산읍 신천리 중 17080964의 지명 요약본 저장 완료
제주특별자치도 서귀포시 성산읍 신천리 중 17080947의 지명 요약본 저장 완료
제주특별자치도 서귀포시 성산읍 신천리 중 17080807의 지명 요약본 저장 완료
제주특별자치도 서귀포시 성산읍 신천리 중 17080816의 

제주특별자치도 서귀포시 효돈순환로 441 중 11621632의 여행지 요약본 저장 완료
제주특별자치도 서귀포시 효돈순환로 441 중 1054016820의 기념품 요약본 저장 완료
제주특별자치도 서귀포시 효돈순환로 441완료
제주특별자치도 제주시 애월읍 애원로 283완료
제주특별자치도 제주시 연북로 111완료
제주특별자치도 제주시 용담3동의 여행지 요약본 저장 완료
제주특별자치도 제주시 용담3동의 여행지 요약본 저장 완료
제주특별자치도 제주시 용담3동완료
제주특별자치도 제주시 애월읍 금성하안길 3완료
제주특별자치도 제주시 서문로5길 1완료
제주특별자치도 제주시 애월읍 광상로 403 주위장소 리스트 존재 X
제주특별자치도 제주시 한경면 용수리 산 60 주위장소 리스트 존재 X
제주특별자치도 제주시 조천읍 조천8길 14-1 주위장소 리스트 존재 X
제주특별자치도 서귀포시 중문관광로 198 (중문동)완료
제주특별자치도 제주시 신산로 82 중 1256878354의 여행지 요약본 저장 완료
제주특별자치도 제주시 신산로 82완료
제주특별자치도 제주시 우도면 우도해안길 84-2완료
제주특별자치도 제주시 한림읍 협재8길 8완료
제주특별자치도 제주시 산록북로 660 중 174369340의 여행지 요약본 저장 완료
제주특별자치도 제주시 산록북로 660완료
제주특별자치도 제주시 관덕로2길 11 주위장소 리스트 존재 X
제주특별자치도 제주시 임항로 111 (건입동)일대완료
제주특별자치도 제주시 명림로 584 (봉개동)의 여행지 요약본 저장 완료
제주특별자치도 제주시 명림로 584 (봉개동)의 여행지 요약본 저장 완료
제주특별자치도 제주시 명림로 584 (봉개동)의 여행지 요약본 저장 완료
제주특별자치도 제주시 명림로 584 (봉개동)의 여행지 요약본 저장 완료
제주특별자치도 제주시 명림로 584 (봉개동)의 여행지 요약본 저장 완료
제주특별자치도 제주시 명림로 584 (봉개동)의 여행지 요약본 저장 완료
제주특별자치도 제주시 명림로 584 (봉개동)완료
제주특별자치도 서귀포시 성산읍

제주특별자치도 제주시 화북1동 4438 주위장소 리스트 존재 X
제주특별자치도 서귀포시 대정읍 동일하모로 158-3완료
한경면 낙수로 97 중 12453098의 여행지 요약본 저장 완료
한경면 낙수로 97 중 21758535의 여행지 요약본 저장 완료
한경면 낙수로 97 중 1594717606의 여행지 요약본 저장 완료
한경면 낙수로 97완료
제주특별자치도 제주시 한경면 조수리의 여행지 요약본 저장 완료
제주특별자치도 제주시 한경면 조수리 중 36311025의 지명 요약본 저장 완료
제주특별자치도 제주시 한경면 조수리 중 38155477의 지명 요약본 저장 완료
제주특별자치도 제주시 한경면 조수리 중 17039459의 지명 요약본 저장 완료
제주특별자치도 제주시 한경면 조수리 중 38161250의 지명 요약본 저장 완료
제주특별자치도 제주시 한경면 조수리 중 1641475411의 지명 요약본 저장 완료
제주특별자치도 제주시 한경면 조수리 중 38161253의 지명 요약본 저장 완료
제주특별자치도 제주시 한경면 조수리 중 38161078의 지명 요약본 저장 완료
제주특별자치도 제주시 한경면 조수리의 여행지 요약본 저장 완료
제주특별자치도 제주시 한경면 조수리 중 17039307의 지명 요약본 저장 완료
제주특별자치도 제주시 한경면 조수리 중 17039335의 지명 요약본 저장 완료
제주특별자치도 제주시 한경면 조수리 중 38161243의 지명 요약본 저장 완료
제주특별자치도 제주시 한경면 조수리 중 38161091의 지명 요약본 저장 완료
제주특별자치도 제주시 한경면 조수리 중 38161245의 지명 요약본 저장 완료
제주특별자치도 제주시 한경면 조수리 중 38161248의 지명 요약본 저장 완료
제주특별자치도 제주시 한경면 조수리 중 38161095의 지명 요약본 저장 완료
제주특별자치도 제주시 한경면 조수리 중 38161099의 지명 요약본 저장 완료
제주특별자치도 제주시 한경면 조수리 중 38161256의 지명 요약본 저장 완료
제주특별자치도 제주시 한경면

제주특별자치도 제주시 추자면 묵리 중 1614998389의 여행지 요약본 저장 완료
제주특별자치도 제주시 추자면 묵리 중 1028163711의 지명 요약본 저장 완료
제주특별자치도 제주시 추자면 묵리 중 31348134의 지명 요약본 저장 완료
제주특별자치도 제주시 추자면 묵리 중 17040977의 지명 요약본 저장 완료
제주특별자치도 제주시 추자면 묵리 중 38158121의 지명 요약본 저장 완료
제주특별자치도 제주시 추자면 묵리 중 38158123의 지명 요약본 저장 완료
제주특별자치도 제주시 추자면 묵리완료
제주특별자치도 제주시 구좌읍 평대7길 34완료
제주특별자치도 제주시 구좌읍 중산간동로  검색결과는 존재하지않음
제주특별자치도 제주시 애월읍 금성하안길 3완료
제주특별자치도 서귀포시 1100로 453-95 의 스포츠 요약본 저장 완료
제주특별자치도 서귀포시 1100로 453-95완료
제주특별자치도 제주시 화북동완료
제주특별자치도 제주시 삼양1동 1938-9 주위장소 리스트 존재 X
제주특별자치도 제주시 우도면 우도봉길 47완료
제주특별자치도 제주시 애월읍 상귀서길 95 주위장소 리스트 존재 X
제주시 한경면 청수서5길 63 중 17037892의 여행지 요약본 저장 완료
제주시 한경면 청수서5길 63완료
제주특별자치도 제주시 구좌읍 월정리 1400-14 중 38590206의 여행지 요약본 저장 완료
제주특별자치도 제주시 구좌읍 월정리 1400-14완료
제주시 한림읍 한림해안로 274 주위장소 리스트 존재 X
제주특별자치도 제주시 한림읍 광산로 942 중 20947594의 여행지 요약본 저장 완료
제주특별자치도 제주시 한림읍 광산로 942 중 31149186의 여행지 요약본 저장 완료
제주특별자치도 제주시 한림읍 광산로 942완료
제주특별자치도 서귀포시 돈내코로 120완료
제주특별자치도 제주시 애월읍 금성1길 29-3완료
제주특별자치도 제주시 한경면 한경해안로 470완료
제주특별자치도 제주시 용담3동의 여행지 요약본 저장 완료
제주특별자치도 제주시 용담3동

In [68]:
tour_result =pd.DataFrame(tour_result)

In [71]:
tour_result.drop_duplicates(subset='id',ignore_index=True,inplace=True)

In [85]:
tour_result.to_csv('../data/naver_crawling/tour_data_raw.csv',index=False)

In [84]:
drop_lsts = ["isSite","type","rCode","posExact","address","addressAbbr","roadAddr","isCallLink","adult","endPageUrl","mobileEndPageUrl","ktCallMd","ppc","imageModDate","displayCategory","category","categoryPaths","previewImages","streetPanorama","skyPanorama","insidePanorama","interiorPanorama","indoorPanorama","poiInfo","entranceCoords","theme","petrolInfo","marker","markerSelected","datalab","dynamicData","markerLabel","isParkingSupported","indoor"]


,id,isSite,name,type,rCode,x,y,posExact,address,fullAddress,...,michelinGuide,broadcastInfo,marker,markerSelected,datalab,reviewCount,dynamicData,markerLabel,isParkingSupported,poiInfo
0,32152539,True,시청에있는멀티방,drt,14110104,126.529667,33.498444,True,제주 제주시 이도이동 1767-15,제주특별자치도 제주시 이도이동 1767-15,...,None,None,nres://marker/00079-00034,nres://marker/00079-00197,None,0,None,"{'text': None, 'style': 'default'}",False,NaN
1,1744843816,True,핸드메이드 라이프,drt,1413025021,126.261712,33.224058,True,제주 서귀포시 대정읍 상모리 3169-1,제주특별자치도 서귀포시 대정읍 상모리 3169-1,...,None,None,nres://marker/00078-00034,nres://marker/00078-00197,None,27,None,"{'text': None, 'style': 'default'}",False,NaN
2,17078730,True,궁대오름,drt,1413025925,126.823141,33.435542,True,제주 서귀포시 성산읍 수산리 4711,제주특별자치도 서귀포시 성산읍 수산리 4711,...,None,None,nres://marker/00076-00139,nres://marker/00076-00303,None,20,None,"{'text': None, 'style': 'default'}",False,NaN
3,1317124374,True,헌마공신김만일기념관,drt,1413025323,126.690724,33.336096,True,제주 서귀포시 남원읍 한남리 992,제주특별자치도 서귀포시 남원읍 한남리 992,...,None,None,nres://marker/00079-00379,nres://marker/00079-00349,None,26,None,"{'text': None, 'style': 'default'}",False,"{'hasRelation': True, 'hasRoad': False, 'hasLa..."
4,1033492321,True,강병대교회,drt,1413025021,126.257388,33.226664,True,제주 서귀포시 대정읍 상모리 3846,제주특별자치도 서귀포시 대정읍 상모리 3846,...,None,None,nres://marker/00076-00045,nres://marker/00076-00208,None,19,None,"{'text': None, 'style': 'default'}",False,"{'hasRelation': False, 'hasRoad': False, 'hasL..."


In [75]:
tour_result.head()

,id,isSite,name,type,rCode,x,y,posExact,address,fullAddress,...,michelinGuide,broadcastInfo,marker,markerSelected,datalab,reviewCount,dynamicData,markerLabel,isParkingSupported,poiInfo
0,32152539,True,시청에있는멀티방,drt,14110104,126.529667,33.498444,True,제주 제주시 이도이동 1767-15,제주특별자치도 제주시 이도이동 1767-15,...,None,None,nres://marker/00079-00034,nres://marker/00079-00197,None,0,None,"{'text': None, 'style': 'default'}",False,NaN
1,1744843816,True,핸드메이드 라이프,drt,1413025021,126.261712,33.224058,True,제주 서귀포시 대정읍 상모리 3169-1,제주특별자치도 서귀포시 대정읍 상모리 3169-1,...,None,None,nres://marker/00078-00034,nres://marker/00078-00197,None,27,None,"{'text': None, 'style': 'default'}",False,NaN
2,17078730,True,궁대오름,drt,1413025925,126.823141,33.435542,True,제주 서귀포시 성산읍 수산리 4711,제주특별자치도 서귀포시 성산읍 수산리 4711,...,None,None,nres://marker/00076-00139,nres://marker/00076-00303,None,20,None,"{'text': None, 'style': 'default'}",False,NaN
3,1317124374,True,헌마공신김만일기념관,drt,1413025323,126.690724,33.336096,True,제주 서귀포시 남원읍 한남리 992,제주특별자치도 서귀포시 남원읍 한남리 992,...,None,None,nres://marker/00079-00379,nres://marker/00079-00349,None,26,None,"{'text': None, 'style': 'default'}",False,"{'hasRelation': True, 'hasRoad': False, 'hasLa..."
4,1033492321,True,강병대교회,drt,1413025021,126.257388,33.226664,True,제주 서귀포시 대정읍 상모리 3846,제주특별자치도 서귀포시 대정읍 상모리 3846,...,None,None,nres://marker/00076-00045,nres://marker/00076-00208,None,19,None,"{'text': None, 'style': 'default'}",False,"{'hasRelation': False, 'hasRoad': False, 'hasL..."


In [74]:
#리뷰 graphql로 요청 크롤링 
url4 = 'https://pcmap-api.place.naver.com/graphql'
data = [
    #0번째 인덱스
    {
        'operationName':'getVisitorReviewPhotosInVisitorReviewTab',
        'query':'query getVisitorReviewPhotosInVisitorReviewTab($businessId: String!, $businessType: String, $page: Int, $display: Int, $theme: String, $item: String) {\n  visitorReviews(input: {businessId: $businessId, businessType: $businessType, page: $page, display: $display, theme: $theme, item: $item, isPhotoUsed: true}) {\n    items {\n      id\n      rating\n      author {\n        id\n        nickname\n        from\n        imageUrl\n        objectId\n        url\n        __typename\n      }\n      body\n      thumbnail\n      media {\n        type\n        thumbnail\n        __typename\n      }\n      tags\n      status\n      visited\n      originType\n      item {\n        name\n        code\n        options\n        __typename\n      }\n      businessName\n      isFollowing\n      visitCount\n      votedKeywords {\n        code\n        iconUrl\n        iconCode\n        displayName\n        __typename\n      }\n      __typename\n    }\n    starDistribution {\n      score\n      count\n      __typename\n    }\n    hideProductSelectBox\n    total\n    __typename\n  }\n}\n',
        'variables':{
            'businessId':'',
            'businessType':'place',
            'display':20,
            'page':1
        }
        
    },
    #1번째 인덱스
    {
        'operationName':'getVisitorReviews',
        'query': 'query getVisitorReviews($input: VisitorReviewsInput) {\n  visitorReviews(input: $input) {\n    items {\n      id\n      rating\n      author {\n        id\n        nickname\n        from\n        imageUrl\n        objectId\n        url\n        review {\n          totalCount\n          imageCount\n          avgRating\n          __typename\n        }\n        theme {\n          totalCount\n          __typename\n        }\n        __typename\n      }\n      body\n      thumbnail\n      media {\n        type\n        thumbnail\n        __typename\n      }\n      tags\n      status\n      visitCount\n      viewCount\n      visited\n      created\n      reply {\n        editUrl\n        body\n        editedBy\n        created\n        replyTitle\n        __typename\n      }\n      originType\n      item {\n        name\n        code\n        options\n        __typename\n      }\n      isFollowing\n      language\n      highlightOffsets\n      translatedText\n      businessName\n      receiptInfoUrl\n      __typename\n    }\n    starDistribution {\n      score\n      count\n      __typename\n    }\n    hideProductSelectBox\n    total\n    __typename\n  }\n}\n',
        'variables':{
            'input':{
                'bookingBusinessId':None,
                'businessId':'',
                'businessType':'place',
                'cidList':["223267","226396","231957"],
                'display':3,
                'includeContent':True,
                'page':1
            }
        }
    },
    #2번째 인덱스
    {
       'operationName':'getVisitorReviewStats',
        'query':'query getVisitorReviewStats($id: String, $itemId: String, $businessType: String = \"place\") {\n  visitorReviewStats(input: {businessId: $id, itemId: $itemId, businessType: $businessType}) {\n    id\n    name\n    apolloCacheId\n    review {\n      avgRating\n      totalCount\n      scores {\n        count\n        score\n        __typename\n      }\n      starDistribution {\n        count\n        score\n        __typename\n      }\n      imageReviewCount\n      authorCount\n      maxSingleReviewScoreCount\n      maxScoreWithMaxCount\n      __typename\n    }\n    analysis {\n      themes {\n        code\n        label\n        count\n        __typename\n      }\n      menus {\n        label\n        count\n        __typename\n      }\n      votedKeyword {\n        totalCount\n        reviewCount\n        userCount\n        details {\n          category\n          code\n          iconUrl\n          iconCode\n          displayName\n          count\n          previousRank\n          __typename\n        }\n        __typename\n      }\n      __typename\n    }\n    visitorReviewsTotal\n    ratingReviewsTotal\n    __typename\n  }\n}\n',
        'variables':{
            'businessType':'place',
            'id':'',
            'itemId':'0'
        }
    },
    #3번째 인덱스
    {
     'operationName':'getVisitorReviewStats',
        'query':'query getVisitorReviewStats($id: String, $itemId: String, $businessType: String = \"place\") {\n  visitorReviewStats(input: {businessId: $id, itemId: $itemId, businessType: $businessType}) {\n    id\n    name\n    apolloCacheId\n    review {\n      avgRating\n      totalCount\n      scores {\n        count\n        score\n        __typename\n      }\n      starDistribution {\n        count\n        score\n        __typename\n      }\n      imageReviewCount\n      authorCount\n      maxSingleReviewScoreCount\n      maxScoreWithMaxCount\n      __typename\n    }\n    analysis {\n      themes {\n        code\n        label\n        count\n        __typename\n      }\n      menus {\n        label\n        count\n        __typename\n      }\n      votedKeyword {\n        totalCount\n        reviewCount\n        userCount\n        details {\n          category\n          code\n          iconUrl\n          iconCode\n          displayName\n          count\n          previousRank\n          __typename\n        }\n        __typename\n      }\n      __typename\n    }\n    visitorReviewsTotal\n    ratingReviewsTotal\n    __typename\n  }\n}\n',
        'variables':{
            'businessType':'place',
            'id':''
        }
    },
    #4번째 인덱스
    {
        'operationName':'getUgcReviewList',
        'query':'query getUgcReviewList($businessId: String) {\n  restaurant(id: $businessId, isNx: false, deviceType: \"mobile\") {\n    fsasReviews {\n      ...FsasReviews\n      __typename\n    }\n    __typename\n  }\n}\n\nfragment FsasReviews on FsasReviewsResult {\n  total\n  maxItemCount\n  items {\n    name\n    type\n    typeName\n    url\n    home\n    id\n    title\n    rank\n    contents\n    bySmartEditor3\n    hasNaverReservation\n    thumbnailUrl\n    thumbnailUrlList\n    thumbnailCount\n    date\n    isOfficial\n    isRepresentative\n    profileImageUrl\n    isVideoThumbnail\n    reviewId\n    authorName\n    createdString\n    __typename\n  }\n  __typename\n}\n',
        'variables':{
            'businessId':''
        }
    },
    #5번째 인덱스
    {
        'operationName':'useBusiness',
        'query':'query useBusiness($id: String, $isNx: Boolean) {\n  restaurant(id: $id, isNx: $isNx, deviceType: \"mobile\") {\n    kinQna {\n      answerCount\n      answerList {\n        detailUrl\n        writeTime\n        contents\n        questionTitle\n        thumbnailUrl\n        dirId\n        __typename\n      }\n      profileUrl\n      __typename\n    }\n    __typename\n  }\n}\n',
        'variables':{
            'id':'',
            'isNx':False
        }
    }
    
]

In [78]:
headers4 = {
    'method': 'POST',
    'scheme': 'https',
    'accept': '*/*',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'ko',
    'content-length': '6443',
    'content-type': 'application/json',
    'cookie': 'NNB=X7N6OU5OFDRWE',
    'sec-ch-ua': '\".Not/A)Brand\";v=\"99\", \"Google Chrome\";v=\"103\", \"Chromium\";v=\"103\"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '\"Windows\"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-site',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'
}

In [79]:
#검색한 요약본을 토대로 리뷰 크롤링
review_datas = []
for index,id_value in enumerate(tour_result['id']):
    search_id = str(id_value)
    #form data 에 아이디값 특정해주기
    data[0]['variables']['businessId'] = str(search_id)
    data[1]['variables']['input']['businessId'] = str(search_id)
    data[2]['variables']['id'] = str(search_id)
    data[3]['variables']['id'] = str(search_id)
    data[4]['variables']['businessId'] = str(search_id)
    data[5]['variables']['id'] = str(search_id)
    time.sleep(random.uniform(3.5,4))
    response4 = requests.post(url4,json=data,headers=headers4)
    if response4.status_code == 200:
        j_text4 = response4.json()
        tmp_dict ={
            'id': search_id,
            'review_datas': j_text4
        }
        review_datas.append(tmp_dict)
        print(f'{index}번째 {id_value} 댓글 조회 성공')
    else: 
        print(f'{index} 번째 {id_value} 조회 실패')
        break
        
    

0번째 32152539 댓글 조회 성공
1번째 1744843816 댓글 조회 성공
2번째 17078730 댓글 조회 성공
3번째 1317124374 댓글 조회 성공
4번째 1033492321 댓글 조회 성공
5번째 17089759 댓글 조회 성공
6번째 17088128 댓글 조회 성공
7번째 17087735 댓글 조회 성공
8번째 16994185 댓글 조회 성공
9번째 17084701 댓글 조회 성공
10번째 17084537 댓글 조회 성공
11번째 17067976 댓글 조회 성공
12번째 12226274 댓글 조회 성공
13번째 13491591 댓글 조회 성공
14번째 13044022 댓글 조회 성공
15번째 1539131369 댓글 조회 성공
16번째 17065061 댓글 조회 성공
17번째 38644345 댓글 조회 성공
18번째 17064757 댓글 조회 성공
19번째 1987691673 댓글 조회 성공
20번째 17085066 댓글 조회 성공
21번째 1071488603 댓글 조회 성공
22번째 17030568 댓글 조회 성공
23번째 1990070640 댓글 조회 성공
24번째 13526015 댓글 조회 성공
25번째 17059754 댓글 조회 성공
26번째 36797503 댓글 조회 성공
27번째 36771178 댓글 조회 성공
28번째 1103132083 댓글 조회 성공
29번째 13293501 댓글 조회 성공
30번째 1494975287 댓글 조회 성공
31번째 1802071455 댓글 조회 성공
32번째 32323100 댓글 조회 성공
33번째 11491304 댓글 조회 성공
34번째 1825235975 댓글 조회 성공
35번째 1880206452 댓글 조회 성공
36번째 1071392259 댓글 조회 성공
37번째 1506089464 댓글 조회 성공
38번째 11797028 댓글 조회 성공
39번째 17089529 댓글 조회 성공
40번째 13139387 댓글 조회 성공
41번째 19551102 댓글 조회 성공
42번째 1330492606 

338번째 36935667 댓글 조회 성공
339번째 1477312816 댓글 조회 성공
340번째 1268398914 댓글 조회 성공
341번째 1205503379 댓글 조회 성공
342번째 1020862610 댓글 조회 성공
343번째 1051505572 댓글 조회 성공
344번째 1519191855 댓글 조회 성공
345번째 1875149782 댓글 조회 성공
346번째 1573461858 댓글 조회 성공
347번째 34761905 댓글 조회 성공
348번째 966468458 댓글 조회 성공
349번째 11491638 댓글 조회 성공
350번째 1355709958 댓글 조회 성공
351번째 17053806 댓글 조회 성공
352번째 21093926 댓글 조회 성공
353번째 1916087538 댓글 조회 성공
354번째 13491577 댓글 조회 성공
355번째 20561776 댓글 조회 성공
356번째 17087660 댓글 조회 성공
357번째 16963385 댓글 조회 성공
358번째 17087854 댓글 조회 성공
359번째 17087764 댓글 조회 성공
360번째 17088240 댓글 조회 성공
361번째 112862112 댓글 조회 성공
362번째 11491425 댓글 조회 성공
363번째 21017260 댓글 조회 성공
364번째 17036722 댓글 조회 성공
365번째 17037121 댓글 조회 성공
366번째 16888973 댓글 조회 성공
367번째 13491311 댓글 조회 성공
368번째 11612628 댓글 조회 성공
369번째 1189851220 댓글 조회 성공
370번째 19584967 댓글 조회 성공
371번째 1242320741 댓글 조회 성공
372번째 32408145 댓글 조회 성공
373번째 13350575 댓글 조회 성공
374번째 20104184 댓글 조회 성공
375번째 1772670001 댓글 조회 성공
376번째 1946415637 댓글 조회 성공
377번째 1438222260 댓글 조회 성공
378번째 14

671번째 17075086 댓글 조회 성공
672번째 19645698 댓글 조회 성공
673번째 838111466 댓글 조회 성공
674번째 809118432 댓글 조회 성공
675번째 19649065 댓글 조회 성공
676번째 1911790757 댓글 조회 성공
677번째 37817998 댓글 조회 성공
678번째 1311494555 댓글 조회 성공
679번째 34581383 댓글 조회 성공
680번째 11816108 댓글 조회 성공
681번째 52672575 댓글 조회 성공
682번째 1487618901 댓글 조회 성공
683번째 1783305721 댓글 조회 성공
684번째 1863461995 댓글 조회 성공
685번째 1232058217 댓글 조회 성공
686번째 1058909066 댓글 조회 성공
687번째 1402348114 댓글 조회 성공
688번째 1323846246 댓글 조회 성공
689번째 21758554 댓글 조회 성공
690번째 1036740319 댓글 조회 성공
691번째 1065308877 댓글 조회 성공
692번째 1166758624 댓글 조회 성공
693번째 1041894659 댓글 조회 성공
694번째 1783123905 댓글 조회 성공
695번째 1981866332 댓글 조회 성공
696번째 1951245784 댓글 조회 성공
697번째 1738277534 댓글 조회 성공
698번째 1732253344 댓글 조회 성공
699번째 1420857547 댓글 조회 성공
700번째 17041252 댓글 조회 성공
701번째 1625029269 댓글 조회 성공
702번째 12226327 댓글 조회 성공
703번째 105300906 댓글 조회 성공
704번째 1113375893 댓글 조회 성공
705번째 35929011 댓글 조회 성공
706번째 1021947485 댓글 조회 성공
707번째 1149387660 댓글 조회 성공
708번째 1171524755 댓글 조회 성공
709번째 17083448 댓글 조회 성공
710번째 1148127

1001번째 17094067 댓글 조회 성공
1002번째 16995531 댓글 조회 성공
1003번째 17009940 댓글 조회 성공
1004번째 1363039847 댓글 조회 성공
1005번째 13542625 댓글 조회 성공
1006번째 1993845938 댓글 조회 성공
1007번째 1394129860 댓글 조회 성공
1008번째 13491243 댓글 조회 성공
1009번째 1834796161 댓글 조회 성공
1010번째 12226192 댓글 조회 성공
1011번째 1496248735 댓글 조회 성공
1012번째 13491584 댓글 조회 성공
1013번째 30997566 댓글 조회 성공
1014번째 1256158608 댓글 조회 성공
1015번째 17026182 댓글 조회 성공
1016번째 1439050448 댓글 조회 성공
1017번째 17037145 댓글 조회 성공
1018번째 20031114 댓글 조회 성공
1019번째 17028328 댓글 조회 성공
1020번째 1539446006 댓글 조회 성공
1021번째 17082282 댓글 조회 성공
1022번째 1737681371 댓글 조회 성공
1023번째 17084763 댓글 조회 성공
1024번째 36773742 댓글 조회 성공
1025번째 17059746 댓글 조회 성공
1026번째 33359625 댓글 조회 성공
1027번째 37784023 댓글 조회 성공
1028번째 31011446 댓글 조회 성공
1029번째 1232134014 댓글 조회 성공
1030번째 38552225 댓글 조회 성공
1031번째 17030472 댓글 조회 성공
1032번째 17084549 댓글 조회 성공
1033번째 35732490 댓글 조회 성공
1034번째 13451641 댓글 조회 성공
1035번째 37677880 댓글 조회 성공
1036번째 32572532 댓글 조회 성공
1037번째 1375769524 댓글 조회 성공
1038번째 1777570381 댓글 조회 성공
1039번째 1949562995 댓글 조회 성공

1322번째 16772992 댓글 조회 성공
1323번째 19882124 댓글 조회 성공
1324번째 16996006 댓글 조회 성공
1325번째 38414793 댓글 조회 성공
1326번째 1122203100 댓글 조회 성공
1327번째 17026431 댓글 조회 성공
1328번째 38162278 댓글 조회 성공
1329번째 38162043 댓글 조회 성공
1330번째 17026306 댓글 조회 성공
1331번째 17026355 댓글 조회 성공
1332번째 38156719 댓글 조회 성공
1333번째 38162275 댓글 조회 성공
1334번째 38162281 댓글 조회 성공
1335번째 17026385 댓글 조회 성공
1336번째 38162041 댓글 조회 성공
1337번째 38162284 댓글 조회 성공
1338번째 38162052 댓글 조회 성공
1339번째 38162294 댓글 조회 성공
1340번째 17067067 댓글 조회 성공
1341번째 36230155 댓글 조회 성공
1342번째 38644299 댓글 조회 성공
1343번째 38163354 댓글 조회 성공
1344번째 38160375 댓글 조회 성공
1345번째 17082892 댓글 조회 성공
1346번째 17083162 댓글 조회 성공
1347번째 17083295 댓글 조회 성공
1348번째 17083342 댓글 조회 성공
1349번째 38155951 댓글 조회 성공
1350번째 38155959 댓글 조회 성공
1351번째 38163363 댓글 조회 성공
1352번째 17083074 댓글 조회 성공
1353번째 17083192 댓글 조회 성공
1354번째 38163346 댓글 조회 성공
1355번째 38155946 댓글 조회 성공
1356번째 17083528 댓글 조회 성공
1357번째 38160384 댓글 조회 성공
1358번째 38155965 댓글 조회 성공
1359번째 13098298 댓글 조회 성공
1360번째 17041405 댓글 조회 성공
1361번째 38644479 댓글 조회 성

In [80]:
review_df =pd.DataFrame(review_datas)

In [82]:
review_df.to_csv('../data/tour_review_datas.csv',index=False)

In [13]:
checklst = ['여행','기념','공방']
result = '여행지'